<a href="https://colab.research.google.com/github/lanlooker/Looker-Tutorials/blob/main/08_Disable_Inactive_Users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Purpose**: This script disables users based on the number of days users have not logged into a Looker instance. 

Please click "Open in Colab" to run the script as a one-off task. The script can be downloaded as a .py file to run inside a server (refer to [this page](https://github.com/looker-open-source/sdk-codegen/blob/main/README.md#configuring-lookerini-or-env
) to learn about `init` file) 

**How to run as a one-off task in Google Colab**: 
1. Provide client ID and secret key in the first cell, and run it
2. Run the remaining cells to call functions. 

**Note**: The script is out of scope for Looker chat support. For questions and comments, please post in Looker Community. 

In [ ]:
!pip install looker_sdk
import looker_sdk
import os

os.environ['LOOKERSDK_BASE_URL'] = 'https://company.looker.com:19999'
os.environ['LOOKERSDK_CLIENT_ID'] = 'foo'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'bar'

#Initialize the SDK
sdk = looker_sdk.init40() 

In [ ]:
def deactivate(auth_method, days):
  '''Deactivate unengaged users who have not logged into a Looker instance in the
  past amount of days. 

  Args:
  auth_method (str): Possible input are 'email', 'embed', 'google' (Google OAuth), 
  'ldap', 'looker_openid', saml', 'oidc', 'totp'
  days(int): Number of days since last attempted UI login
  '''

  '''The all_users() method returns all users in a Looker instance. To use search
  patterns, use search_users(), or search_users_names()'''
  users = sdk.all_users()

  '''In Looker UI, a non-embed user usually has one primary authentication method
  (usually email or SSO). In calling API/SDK call, all authentication methods will
  be returned, with non-active authentication returned as an empty dictionary'''
  auth = "credentials_" + auth_method

  '''A user with a valid authentication method but never logged in has the value 
  for "logged_in_at" as an empty string.''' 
  users_with_login_date = list(filter(lambda user:user[auth] is not None 
                                      and user[auth]["logged_in_at"] != "", users))

  if len(users_with_login_date) == 0:
    return ("No users are using the provided authentication method")
  
  else:
    user_deactivated_count = 0
    user_skip_count = 0
    for user in users_with_login_date:
      last_login_date_str = user[auth]["logged_in_at"][0:10]
      last_login_date = datetime.strptime(last_login_date_str,'%Y-%m-%d')
      days_since_last_login = abs(datetime.now() - last_login_date).days
      if days_since_last_login > days:
        sdk.update_user(user_id = user["id"],body=models40.WriteUser(is_disabled = False))
        user_deactivated_count += 1
        print("Disabled user id {user_id}, {email}, whose last login date is {date}".format(user_id = user["id"], email=user[auth]["email"],date=last_login_date_str))
      else:
        user_skip_count += 1
        print("Skipped user ID {user_id}, {email}, whose last attempted login is {date}".format(user_id = user["id"],email=user[auth]["email"], date=last_login_date_str))

  print("Deactivated {user_deactivated_count} users".format(user_deactivated_count=user_deactivated_count))


# Example call
deactivate(auth_method="email", days=180)